# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([  1.11022302e-16,   1.11022302e-16,  -1.11022302e-16,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -1.38777878e-17,   0.00000000e+00,   2.77555756e-17,
         1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
# ... ENTER YOUR CODE HERE ...
def jacobi(A,b,maxiter):
    # for 2D matrix, return its diagonal
    diag_1d = np.diag(A)
    B = -A.copy()
    # fill diagonal with all zero
    np.fill_diagonal(B, 0)
    
    # for a vector, fill it on a matrix diagonal
    D = np.diag(diag_1d)

    # for a diagonal matrix, its inv is just inverting all elements on diagonal
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b

    x0 = np.ones(n)
    x = x0
    for _ in range(maxiter):
        x = BB @ x + c
    return x

x15=[]
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
for i in range(50):
    x=jacobi(A,b,i)
    x15.append(np.linalg.norm(A@x-b))

x10=[]
A = rndm.uniform(size=(n, n)) + np.diagflat([10]*n)
for i in range(50):
    x=jacobi(A,b,i)
    x10.append(np.linalg.norm(A@x-b))

x5=[]
A = rndm.uniform(size=(n, n)) + np.diagflat([5]*n)
for i in range(50):
    x=jacobi(A,b,i)
    x5.append(np.linalg.norm(A@x-b))

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...

def seidel(A,b,maxiter):
    # for 2D matrix, return its diagonal
    # construct Upper matrix
    U=np.triu(A)
    np.fill_diagonal(U, 0)
    
    # construct D
    diag_1d = np.diag(A)
    # for a vector, fill it on a matrix diagonal
    D = np.diag(diag_1d)
    
    # construct L
    L=np.tril(A)
    
    DL=L
    DL_n1=np.linalg.inv(DL)

    x0 = np.ones(n)
    x = x0
    for _ in range(maxiter):
        x = DL_n1@(b-U@x)
    return x



# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...
def mrs(A,b,maxiter):
    x0 = np.ones(n)
    x = x0
    for _ in range(maxiter):
        r=A@x-b
        tau=r@(A@r)/np.linalg.norm(A@r)
        x=x-tau*r
    return x


